# EM shower reconstruction at SND@LHC

1. __make sure the preprocessing has already been done__

2. __make sure `results` folder exists__

https://arxiv.org/pdf/2002.08722.pdf

In [2]:
# imports from utils.py & net.py
from utils import DataPreprocess, Parameters
#from net import SNDNet, BNN, MyDataset, digitize_signal, digitize_signal_1d

# python
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pylab as plt
import time
from tqdm import tqdm
from IPython import display

# system
import os
import gc  # Gabage collector interface (to debug stuff)
import sys

# ml
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# dl
import torch
import torch.nn as nn

Welcome to JupyROOT 6.18/00


In [3]:
# Test to see if cuda is available or not + listed the CUDA devices that are available
try:
    assert(torch.cuda.is_available())
except:
    raise Exception("CUDA is not available")
    
n_devices = torch.cuda.device_count()
print("CUDA devices available:")

for i in range(n_devices):
    print("\t{}\twith CUDA capability {}".format(torch.cuda.get_device_name      (device=i), 
                                                 torch.cuda.get_device_capability(device=i)))

device = torch.device("cuda", 0)

CUDA devices available:
	Quadro RTX 4000	with CUDA capability (7, 5)
	Quadro RTX 4000	with CUDA capability (7, 5)


In [4]:
# Turn off interactive plotting: for long run it screws up everything
plt.ioff()

In [5]:
DETECTOR_PARAMS = Parameters("SNDatLHC")
DETECTOR_CONFIG = DETECTOR_PARAMS.snd_params[DETECTOR_PARAMS.configuration]

# number of planes of the detector
#NB_PLANE = dict()

#NB_PLANE['scifi']   = len(DETECTOR_CONFIG['SciFi_tracker']        ['TT_POSITIONS'])
#NB_PLANE['up_mu']   = len(DETECTOR_CONFIG['Mu_tracker_upstream']  ['TT_POSITIONS'])
#NB_PLANE['down_mu'] = len(DETECTOR_CONFIG['Mu_tracker_downstream']['TT_POSITIONS'])

## Data processing

Here we load and process __pickle__ files. 

In [6]:
from src.process_pickle import *

In [7]:
DATA_PATH = dict()
DATA_PATH['nuel']  = "~/snd_data/nue"
DATA_PATH['numu']  = "~/snd_data/numu"
DATA_PATH['nutau'] = "~/snd_data/nutau"

EVENTS_PER_FILE = 4000 # todo -> read from the files ?
FILES_NUM       = 8   # MAX=100 / todo -> read from directory ?

In [8]:
#scifi_arr, mu_arr, en_arr = load_dataframes(DETECTOR_PARAMS, 
#                                            DATA_PATH, EVENTS_PER_FILE, FILES_NUM)

In [9]:
#scifi_arr, mu_arr, en_arr = merge_events_arrays(scifi_arr, mu_arr, en_arr)

In [10]:
#en_arr = normalise_target_energy(en_arr)

## Data preparation

Here we prepare (load or, if needed, create) the datasets.

In [11]:
from src.operate_datasets import *

In [26]:
#create_dataset('true', DETECTOR_PARAMS, DATA_PATH, EVENTS_PER_FILE, FILES_NUM)

In [27]:
#create_dataset('sum', DETECTOR_PARAMS, DATA_PATH, EVENTS_PER_FILE, FILES_NUM)

In [28]:
#create_dataset('longitudal', DETECTOR_PARAMS, DATA_PATH, EVENTS_PER_FILE, FILES_NUM)

In [15]:
create_dataset('projection', DETECTOR_PARAMS, DATA_PATH, EVENTS_PER_FILE, FILES_NUM)


Reading the tt_cleared.pkl & y_cleared.pkl files by chunk of CCDIS and NueEElastic
Before Reduction (file ~/snd_data/nue):
  TT_df  : 32000
  MU_df  : 32000
  y_full : 32000
Before Reduction (file ~/snd_data/numu):
  TT_df  : 32000
  MU_df  : 32000
  y_full : 32000
Before Reduction (file ~/snd_data/nutau):


  0%|          | 45/96000 [00:00<03:59, 401.14it/s]

  TT_df  : 32000
  MU_df  : 32000
  y_full : 32000
After Reduction  :

Particle type: nuel
  scifi_arr : 32000
  mu_arr    : 32000
  en_arr    : 32000
Particle type: numu
  scifi_arr : 32000
  mu_arr    : 32000
  en_arr    : 32000
Particle type: nutau
  scifi_arr : 32000
  mu_arr    : 32000
  en_arr    : 32000

combined_scifi_arr : 96000
combined_mu_arr: 96000
combined_en_arr: 96000
projection


100%|██████████| 96000/96000 [02:05<00:00, 763.21it/s] 


In [16]:
# memory troubles!
# be very carefull when using this
### create_dataset('plane', DETECTOR_PARAMS, DATA_PATH)

In [ ]:
!mv *.npz ../snd_data/d_data/new_dataset/

## Models

In [17]:
# Support Vector Regression (SVR)

'''
full_X, full_y = load_dataset('~/snd_data/new_dataset/', 'longitudal')

X_train, y_train, _, _ = split_dataset(full_X, full_y)
# min_clip = 25
# X_train, y_train = clip_dataset(X_train, y_train, min_clip)


from sklearn import svm

reg_svr = svm.SVR(gamma='scale')
#reg_svr = svm.LinearSVR(max_iter=10**5)

reg_svr.fit(X_train, y_train)

score_svr = reg_svr.score(X_train, y_train)

print('SVM: ', score_svr)

y_pred_svr = reg_svr.predict(X_train)

X_sum = X_train.sum(axis=1).reshape(-1,1)
y_sum = y_train.reshape(-1,1)
y_pred_svr = y_pred_svr.reshape(-1,1)


plot_res_vs_energy(X_sum, y_sum, y_pred_svr) 
plot_res_hist(y_sum, y_pred_svr)
plot_2d_energy_hist(X_sum, y_sum, y_pred_svr)
get_scores(y_sum, y_pred_svr)
'''

"\nfull_X, full_y = load_dataset('~/snd_data/new_dataset/', 'longitudal')\n\nX_train, y_train, _, _ = split_dataset(full_X, full_y)\n# min_clip = 25\n# X_train, y_train = clip_dataset(X_train, y_train, min_clip)\n\n\nfrom sklearn import svm\n\nreg_svr = svm.SVR(gamma='scale')\n#reg_svr = svm.LinearSVR(max_iter=10**5)\n\nreg_svr.fit(X_train, y_train)\n\nscore_svr = reg_svr.score(X_train, y_train)\n\nprint('SVM: ', score_svr)\n\ny_pred_svr = reg_svr.predict(X_train)\n\nX_sum = X_train.sum(axis=1).reshape(-1,1)\ny_sum = y_train.reshape(-1,1)\ny_pred_svr = y_pred_svr.reshape(-1,1)\n\n\nplot_res_vs_energy(X_sum, y_sum, y_pred_svr) \nplot_res_hist(y_sum, y_pred_svr)\nplot_2d_energy_hist(X_sum, y_sum, y_pred_svr)\nget_scores(y_sum, y_pred_svr)\n"

## Run models

In [37]:
!jupyter nbconvert --to notebook --inplace --execute regression_*.ipynb

[NbConvertApp] Converting notebook regression_full_sum.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 168125 bytes to regression_full_sum.ipynb
[NbConvertApp] Converting notebook regression_plane_sum.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 231943 bytes to regression_plane_sum.ipynb
[NbConvertApp] Converting notebook regression_projections.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 235004 bytes to regression_projections.ipynb
[NbConvertApp] Converting notebook regression_true_hits.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 218294 bytes to regression_true_hits.ipynb


In [44]:
!jupyter nbconvert --to notebook --inplace --execute nn_*.ipynb --ExecutePreprocessor.timeout=180

[NbConvertApp] Converting notebook nn_full_sum.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 196209 bytes to nn_full_sum.ipynb
[NbConvertApp] Converting notebook nn_plane_sum.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 274110 bytes to nn_plane_sum.ipynb
[NbConvertApp] Converting notebook nn_projections.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 246637 bytes to nn_projections.ipynb


In [45]:
!jupyter nbconvert --to notebook --inplace --execute bnn_*.ipynb

[NbConvertApp] Converting notebook bnn_full_sum.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 240058 bytes to bnn_full_sum.ipynb
[NbConvertApp] Converting notebook bnn_plane_sum.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 248118 bytes to bnn_plane_sum.ipynb


## Compare metrics

In [46]:
from src.model_evaluation import *

In [56]:
def present_scores(df):
    df = df.reindex(sorted(df.columns), axis=1)
    first_col = df.pop('Score')
    df.insert(0, 'Score', first_col)

    return df

In [57]:
present_scores(collect_all_scores())

,Score,BNN-on-full-sum,BNN-on-plane-sums,L2-on-full-sum,L2-on-plane-sums,L2-on-projections,L2-on-true-hits,NN-on-full-sum,NN-on-plane-sums,NN-on-projections
0,explained_variance_score,-6.614871,-0.075949,0.013989,0.279462,0.352173,0.127250,0.040829,0.177364,0.366390
1,max_error,15.184106,3.905588,0.962614,1.035705,1.083276,0.999070,1.035946,2.611687,1.052366
2,mean_absolute_error,0.227494,0.133608,0.140651,0.113108,0.107232,0.125568,0.127268,0.117873,0.106135
3,mean_squared_error,0.244339,0.034678,0.031907,0.023052,0.020750,0.027922,0.031396,0.026318,0.020295
4,median_absolute_error,0.131900,0.108528,0.127065,0.092548,0.087339,0.107848,0.099604,0.094789,0.087308
5,r2_score,-6.637339,-0.083925,0.002681,0.279462,0.352172,0.127250,0.018643,0.177362,0.366379
6,avg_resolution,1.120973,1.818890,2.375080,1.498581,1.400614,1.872374,1.551465,1.489076,1.400872
7,std_resolution,8.198869,7.505279,8.835667,5.833365,5.602714,7.319886,6.081390,5.644837,5.425789


In [58]:
present_scores(collect_all_scores(model_evaluation.TEST_SCORES_DIR))

,Score,BNN-on-full-sum,BNN-on-plane-sums,L2-on-full-sum,L2-on-plane-sums,L2-on-projections,L2-on-true-hits,NN-on-full-sum,NN-on-plane-sums,NN-on-projections
0,explained_variance_score,-3.174983,-0.071997,0.015207,0.277664,0.337756,0.130124,0.045816,0.123731,0.355991
1,max_error,6.641919,2.012484,0.907420,0.856899,0.805914,0.894357,0.943592,3.245611,0.933178
2,mean_absolute_error,0.212279,0.134610,0.140498,0.113324,0.107791,0.125348,0.126719,0.118628,0.106765
3,mean_squared_error,0.137757,0.034476,0.031751,0.023037,0.020905,0.027743,0.031179,0.027947,0.020337
4,median_absolute_error,0.139006,0.109266,0.126730,0.092459,0.087961,0.107367,0.098393,0.095027,0.087584
5,r2_score,-3.319428,-0.081010,0.004419,0.277664,0.337663,0.130113,0.022369,0.123714,0.355646
6,avg_resolution,0.723605,1.724431,2.247229,1.406165,1.324153,1.769530,1.460338,1.406977,1.341442
7,std_resolution,5.612163,5.691439,6.652317,4.396246,4.061178,5.507832,4.717037,4.327076,4.023060


## Stochastic regression

In [18]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO, Predictive

In [19]:
full_X, full_y = load_dataset('~/snd_data/new_dataset/', 'sum')
min_clip = 25

X_train, y_train = dataset_split(full_X, full_y)
X_train, y_train = dataset_clip(X_train, y_train, min_clip)

NameError: name 'dataset_split' is not defined

In [ ]:
X_train = torch.tensor(X_train).float().reshape(-1,1)
y_train = torch.tensor(y_train).float().reshape(-1)

In [ ]:
from pyro.distributions import constraints


class BayesianRegression(PyroModule):
    def __init__(self):
        super().__init__()
        self.linear = PyroModule[nn.Linear](1, 1)
        
        # prior over parameters
        self.linear.weight = PyroSample(dist.Normal(1e-4, 5e-5).expand([1, 1]).to_event(2))
        self.linear.bias   = PyroSample(dist.Normal(1e-1, 5e-2).expand([1]).to_event(1))
        return
    
    def forward(self, x, y=None):
        # energy (from the model)
        mean = self.linear(x).squeeze(-1)
        
        # noise (learnable)
        sigma = pyro.sample("sigma", dist.Uniform(0., 1.)) 
        
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        
        return mean

In [ ]:
def model(x, y=None):
    bias   = pyro.sample("bias",   dist.Normal(1e-4, 1e-4))
    weight = pyro.sample("weight", dist.Normal(0.1, 0.1))

    mean = bias + weight * x # + mu_weight * muon_hits
    mean = mean.squeeze(-1)
    sigma = pyro.sample("sigma", dist.Uniform(0., 0.1)) 

    with pyro.plate("data", x.shape[0]):
        pyro.sample("obs", dist.Normal(mean, sigma), obs=y)

    return mean


def guide(x, y=None):
    b_loc   = pyro.param("b_loc",   torch.tensor(1e-4), constraint=constraints.positive)
    b_scale = pyro.param("b_scale", torch.tensor(1e-4), constraint=constraints.positive)

    w_loc   = pyro.param("w_loc",   torch.tensor(0.1), constraint=constraints.positive)
    w_scale = pyro.param("w_scale", torch.tensor(0.1), constraint=constraints.positive)

    bias =   pyro.sample("bias",   dist.Normal(b_loc, b_scale))
    weight = pyro.sample("weight", dist.Normal(w_loc, w_scale))

    #sigma_loc = pyro.param('sigma_loc', torch.tensor(5.))
    sigma = pyro.sample("sigma", dist.Normal(0.05, 0.005))

    mean = bias + weight * x
    
    return mean

In [ ]:
reg_model = model # BayesianRegression()
reg_guide = guide # AutoDiagonalNormal(model)

num_steps = 50
initial_lr = 1.0
gamma = 0.5  # final learning rate will be gamma * initial_lr
lrd = gamma ** (1 / num_steps)

adam = pyro.optim.ClippedAdam({"lr": initial_lr, 'lrd': lrd})

#adam = pyro.optim.ClippedAdam({"lr": 1.0, "lrd": 0.5})
svi = SVI(reg_model, reg_guide, adam, loss=Trace_ELBO())
num_iterations = 200

pyro.clear_param_store()
loss_arr = []


for j in range(num_iterations):
    loss = svi.step(X_train, y_train)

    loss_arr.append(loss)
        
    if j % 25 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(X_train)))

In [ ]:
plt.plot(loss_arr)
plt.yscale('log')
plt.xlabel('epoch')
plt.ylabel('ELBO loss')
plt.show()

In [ ]:
#guide.requires_grad_(False)

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

In [ ]:
def summary(samples):
    site_stats = {}
    for site_name, values in samples.items():
        site_stats[site_name] = {
            "mean": torch.mean(values, 0),
            "std" : torch.std (values, 0),
            "5%"  : values.kthvalue(int(len(values) * 0.05), dim=0)[0],
            "95%" : values.kthvalue(int(len(values) * 0.95), dim=0)[0],
        }
        
    return site_stats


predictive = Predictive(model, guide=guide, num_samples=800)
samples = predictive(X_train)
pred_summary = summary(samples)

In [ ]:
pred_summary

In [ ]:
mu = pred_summary["obs"]
y = pred_summary["obs"]
predictions = pd.DataFrame({
    "mu_mean": mu["mean"],
    "mu_perc_5": mu["5%"],
    "mu_perc_95": mu["95%"],
})

In [ ]:
plt.plot(X_train[:,0], predictions['mu_mean'].to_numpy(), 'o')
plt.plot(X_train[:,0], predictions['mu_perc_5'].to_numpy(), 'o')
plt.plot(X_train[:,0], predictions['mu_perc_95'].to_numpy(), 'o')
plt.show()

In [ ]:
predictions['mu_mean']

In [ ]:
def plot_2d_energy_hist(X_arr, y_true, predictions):
    fig, ax = plt.subplots(figsize=(8,6))

    y_pred_min  = predictions["mu_perc_5"].to_numpy()
    y_pred_mean = predictions["mu_mean"]  .to_numpy()
    y_pred_max  = predictions["mu_perc_95"].to_numpy()
    
    print(X_arr.shape)
    print(y_true.shape)
    
    hist = ax.hist2d(X_arr[:,0].numpy(), y_true.numpy(), 
                     bins=100, norm=mpl.colors.LogNorm(), vmax=150)
    
    plt.ylim(0)
    plt.xlabel('pixel sum')
    plt.ylabel('normalised energy')

    #plt.axvline(x=min_clip, c='m', alpha=0.9, label='Min clip ' + str(min_clip))
    ax.plot(X_arr[:, 0], y_pred_mean, 'g.', alpha=0.3, label='L2 w. clipped data')
    ax.plot(X_arr[:, 0], y_pred_min, 'r.', marker='.', alpha=0.3, label='5%')
    ax.plot(X_arr[:, 0], y_pred_max, 'b.', marker='.', alpha=0.3, label='95%')
    #ax.fill_between(X_arr[:, 0], y_pred_min, y_pred_max,  alpha=0.3, color='deeppink')

    #cbar = fig.colorbar(hist[3], ax=ax)
    #|cbar.set_label('# of particles')

    plt.legend(loc='lower right')
    plt.show()

In [ ]:
plot_2d_energy_hist(X_train, y_train, predictions) 